16940번 BFS 스페셜 저지 <span style="color:red">실패</span> - 2025.04.08*
 - defalutdict 설명글 https://dongdongfather.tistory.com/69

In [ ]:
# 사진에서도 볼 수 있듯이 부모-자식 간의 관계도 고려해야함
# A 노드와 연결된 노드들의 방문 순서는 정해져있지 않다.
# 하지만 만약 A와 연결된 노드 중에서 B를 처음으로 방문했다면 A와 연결된 노드를 모두 방문하고 난 후에는 B와 연결된 노드들을 가장 먼저 탐색해야 한다.
# 입력으로 주어진 BFS 방문순서가 위의 조건에 부합하는지 확인해야 한다.
import sys
from collections import deque, defaultdict
# input = sys.stdin.readline

def bfs():
    q = deque()
    q.append(1) # BFS 1부터 시작
    visited[1] = True

    while q:
        x = q.popleft()

        for next in graph[x]:
            if not visited[next]:
                visited[next] = True
                children[x].append(next) # 딕셔너리에 부모-자식 관계 저장
                q.append(next)

n = int(input())
graph = [[] for _ in range(n + 1)]
visited = [False for _ in range(n + 1)]
for _ in range(n - 1):
    a, b = map(int, input().split())
    graph[a].append(b)
    graph[b].append(a)
ans = list(map(int, input().split()))
children = defaultdict(list)

bfs()
# print(children) # defaultdict(<class 'list'>, {1: [2, 3], 2: [4]})

if ans[0] != 1:
    print(0)
else:
    q = deque()
    q.append(1) # 1 = ans[0]
    idx = 1 # ans[0] 헀으니까 1부터 시작

    while q:
        x = q.popleft()

        a = set(children[x]) # a에 x의 자식노드 저장
        b = ans[idx:idx + len(a)] # 그 depth만큼 슬라이싱 -> x의 자식노드 자르기
        q.extend(b) # 다음 탐색할 노드로 저장
        b = set(b) # 비교 위해서 set으로 변환
        idx += len(a) # 다음 자식노드 탐색하기 위해 idx 늘려주기
        
        if a != b: # 둘의 집합이 다르다면
            print(0)
            break
    else:
        print(1)

16964번 DFS 스페셜 저지 <span style="color:red">실패</span> - 2025.04.09*
 - https://blog.joonas.io/94
 - https://vixxcode.tistory.com/29

In [ ]:
# 올바른 DFS방문 순서가 되기 위해서는 자식이 부모보다 먼저 나오는 경우가 있어서는 안된다.
# 어떤 순서 i번째에 등장하는 노드 x의 깊이는 어떤 순서 i+x의 서브트리의 크기까지는 전부 그 노드의 자식이다.
# 1. DFS에서 각 노드의 depth, 자신을 시작으로 하는 트리의 크기를 저장한다.
# 2. DFS 종료 후 ans를 순차적으로 탐색하며
# 2-1. 자신의 위치(i) + 자신을 시작으로하는 트리의 크기(node_size[tmp])를 한다.
# 2-2. 2-1에서 구한 수와 자신의 depth를 비교한다 (자신보다 크면 형제 or 자식이 아닌 부모 노드이기 때문에 break)
import sys
# input = sys.stdin.readline

def dfs(start, depth): # 1
    if visited[start]:
        return 0
    visited[start] = True
    
    size = 1 # 최소 크기는 1 -> 자기 자신
    node_depth[start] = depth # 각 노드의 depth 저장

    for next in graph[start]: # 각 노드를 시작으로 하는 트리의 크기 저장
        size += dfs(next, depth + 1)
    node_size[start] = size

    return size


n = int(input())
graph = [[] for _ in range(n + 1)]
for _ in range(n - 1):
    a, b = map(int, input().split())
    graph[a].append(b)
    graph[b].append(a)

node_depth = [0 for _ in range(n + 1)]
node_size = [0 for _ in range(n + 1)]
visited = [False for _ in range(n + 1)]
ans = list(map(int, input().split()))

if ans[0] != 1:
    print(0)
    sys.exit()
else:
    dfs(1, 0)
    for i in range(1, n): # 2
        tmp = ans[i]
        # 크기가 1이면 가장 마지막에 있는 노드이고, i + node_size가 n보다 크거나 같아지면 IndexError 발생!
        # ex) 1 - 2 - 3 - 4 -> 1 + 1 = 5 (IndexError)
        if node_size[tmp] == 1 or i + node_size[tmp] >= n:
            continue
        next = ans[i + node_size[tmp]] # 2-1
        if node_depth[next] > node_depth[tmp]: # 2-2
            print(0)
            sys.exit()
    
    print(1)

2146번 다리 만들기 <span style="color:green">성공</span> - 2025.04.10

In [ ]:
# ## numbering_bfs() ##
# 1. 먼저 섬에 따라서 라벨링 (-1부터 시작해서 -- 하면서)
# 1-1. 종료조건은 1일 때만 BFS

# ## building_bfs() ##
# 2. 각 섬의 요소 하나하나마다 0일때만 BFS 할 수 있도록
# 2-1. 다음 노드가 0이면 최단경로 탐색할 때 처럼 ++ 하면서
# 2-2. 종료조건은 다음 노드가 음수이고 시작 섬 번호와 다를 때
# 2-3. 종료하는 순간에 값이 0보다 크면 max(ans, 종료순간 값)
import sys
from collections import deque

def numbering_island(x, y, num):
    q = deque()
    q.append((x, y))

    while q:
        x, y = q.popleft()

        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]

            if 0 <= nx < n and 0 <= ny < n and graph[nx][ny] == 1: # 1-1 바다 말고 섬일때만
                graph[nx][ny] = num
                q.append((nx, ny))

def building_bridge(x, y):
    global ans

    q = deque()
    q.append((x, y))

    break_flag = False
    while q:
        x, y = q.popleft()

        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]

            if 0 <= nx < n and 0 <= ny < n:
                if tmp[nx][ny] == 0: # 2-1. 0일때만 (바다 일때만)
                    tmp[nx][ny] = tmp[x][y] + 1
                    q.append((nx, ny))
                # 2-2. 만약 음수를 만났는데 (섬인데) 시작 섬번호가 아니면 break    
                elif tmp[nx][ny] < 0 and tmp[nx][ny] != origin_num: 
                    ans = min(ans, tmp[x][y])
                    break_flag = True
                    break
        if break_flag:
            break

n = int(input())
graph = [list(map(int, input().split())) for _ in range(n)]

dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

num = -1
for i in range(n): # 모든 섬에 대해서 넘버링
    for j in range(n):
        if graph[i][j] == 1:
            graph[i][j] = num
            numbering_island(i, j, num)
            # 섬 하나 통째로 넘버링 했으면 다음 섬 번호 -- 해주기
            num -= 1

ans = sys.maxsize
for i in range(n):
    for j in range(n):
        if graph[i][j] < 0: # 섬에 대해서만 실행 (섬 번호는 음수니까)
            origin_num = graph[i][j] # 현재 섬 번호 저장 (다른 섬이랑 안헷갈리게 <- 종료조건이므로)
            tmp = [graph[i][:] for i in range(n)] # 다리 그렸던던거 초기화
            tmp[i][j] = 0
            building_bridge(i, j)

print(ans)